# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-35c2113d62-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Olympic Games explorative search") 

Consider the following exploratory information need:

> investigate the Olympic Games and find the main BGPs related to these events. Find all the editions of the Summer and Winter Games, compare the sports and disciplines that belong to the games, records and countries.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q1189`    | Usain Bolt    | node      |





Also consider

```
wd:Q1189 ?p ?obj .
```

is the BGP to retrieve all **properties of Usain Bolt**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for Olympic Games

2. Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

3. Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

4. Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

5. Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

6. Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

7. Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

8. Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal). 

## Task 1
Identify the BGP for Olympic Games

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1189 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'de

In [4]:
# participations of Usain Bolt
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1189 wdt:P1344 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q55256167'), ('name', "2013 Zürich Weltklasse – men's 100 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55256874'), ('name', "2012 Lausanne Athletissima – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55256933'), ('name', "2012 Zürich Weltklasse – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55257847'), ('name', "2010 Lausanne Athletissima – men's 100 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55258884'), ('name', "2009 Lausanne Athletissima – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55259143'), ('name', "2009 Zürich Weltklasse – men's 100 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55259163'), ('name', "2009 Zürich Weltklasse – men's 4 × 100 metres relay")]
[('obj', 'http://www.wikidata.org/entity/Q55259611'), ('name', "2008 Zürich Weltklasse – men's 100 metres")]
[('obj', 'http://www.wikidata.org/entity/Q55259648'), ('name', "2008 Lausanne Athletissima – men's

In [6]:
# properties of athletics 4x100 at the 2012 Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1144200 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'ht

In [7]:
# exploring part of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1144200 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q185262'), ('name', 'athletics at the 2012 Summer Olympics')]
1


In [8]:
# properties of athletics at the 2012 Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q185262 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p'

In [9]:
# exploring part of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q185262 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
1


In [10]:
# properties of 2012 Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8577 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1619'), ('name', 'date of official opening')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID')]
[('p', 'htt

In [11]:
# exploring instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q8577 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
1


In [12]:
# properties of Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q159821 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P269'), ('name', 'IdRef ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), (

In [13]:
# exploring subclass of for Summer Olympics
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q159821 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


Final query for this task

In [14]:
# find Olympic Games BGP
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q159821 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


## Task 2
Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

In [15]:
# properties of Olympic Games
queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q5389 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('name', 'flag')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1807

In [16]:
# find all Summer Olympic editions
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {
   # bind something
   ?editions wdt:P31 wd:Q159821 .
   # get the label
   ?editions sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8438'), ('name', '1972 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8444'), ('name', '1976 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8470'), ('name', '1988 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8488'), ('name', '1992 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8531'), ('name', '1996 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8544'), ('name', '2000 Summer Olympics')]
[('editions

In [2]:
# find all Summer Olympic editions before current time (wrong)
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {
   # bind something
   ?editions wdt:P31 wd:Q159821 .
   # get the label
   ?editions sc:name ?name.
   FILTER(xsd:gYear(SUBSTR(?name, 1, 4)) < "2022"^^xsd:gYear)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [17]:
# exploring point in time and edition number of 2020 Summer Olympic Games (wrong)
queryString = """
SELECT DISTINCT ?point_in_time ?pit_name ?edition_number ?ed_name
WHERE {
   # bind something
   wd:Q181278 wdt:P585 ?point_in_time ;
              wdt:P393 ?edition_number .
   # get the label
   ?point_in_time sc:name ?pit_name.
   ?edition_number sc:name ?ed_name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [18]:
# exploring point in time 2020 Summer Olympic Games (wrong)
queryString = """
SELECT DISTINCT ?point_in_time ?pit_name
WHERE {
   # bind something
   wd:Q181278 wdt:P585 ?point_in_time .
   # get the label
   ?point_in_time sc:name ?pit_name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [26]:
# exploring point in time 2020 Summer Olympic Games
queryString = """
SELECT DISTINCT ?point_in_time
WHERE {
   # bind something
   wd:Q181278 wdt:P585 ?point_in_time .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('point_in_time', '2021-01-01T00:00:00Z')]
1


In [25]:
# exploring edition number of 2020 Summer Olympic Games
queryString = """
SELECT DISTINCT ?obj
WHERE {
   
   wd:Q181278 wdt:P393 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '32')]
1


In [20]:
# find properties of 2020 Summer Olympic Games
queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q181278 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P1451'), ('name', 'motto text')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('name', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2130'), ('name', 'cost')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 

In [ ]:
# find all Summer Olympic editions filtering out future editions (wrong)
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {

   ?editions wdt:P31 wd:Q159821 .
   # get the label
   ?editions sc:name ?name.
   FILTER(substr(?name, 1, 4) < "2021")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [27]:
# find start time of 2020 Summer Olympic Games
queryString = """
SELECT DISTINCT ?obj
WHERE {
   
   wd:Q181278 wdt:P580 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '2021-07-24T00:00:00Z')]
1


In [31]:
# find all Summer Olympic editions filtering out future editions
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {

   ?editions wdt:P31 wd:Q159821 ;
            wdt:P393 ?ed_number.
   # get the label
   ?editions sc:name ?name.
   FILTER(?ed_number <= "32")
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8143'), ('name', '1932 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8368'), ('name', '1940 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8376'), ('name', '1944 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8403'), ('name', '1948 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8407'), ('name', '1952 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8411'), ('name', '1956 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics')]
[('editions',

In [32]:
# find all Summer Olympic editions filtering out future editions (wrong)
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {

   ?editions wdt:P31 wd:Q159821 ;
            wdt:P580 ?start_time.
   # get the label
   ?editions sc:name ?name.
   FILTER(?start_time <= "2021-07-24T00:00:00Z")
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q995653'), ('name', '2024 Summer Olympics')]
2


In [33]:
# find all Summer Olympic editions filtering out future editions
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {

   ?editions wdt:P31 wd:Q159821 .
   OPTIONAL{?editions wdt:P393 ?ed_number}
   # get the label
   ?editions sc:name ?name.
   FILTER(?ed_number <= "32")
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8143'), ('name', '1932 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8368'), ('name', '1940 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8376'), ('name', '1944 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8403'), ('name', '1948 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8407'), ('name', '1952 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8411'), ('name', '1956 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics')]
[('editions',

In [34]:
# find start time of 1928 Summer Olympic Games
queryString = """
SELECT DISTINCT ?obj
WHERE {
   
   wd:Q8138 wdt:P580 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1928-07-28T00:00:00Z')]
1


In [35]:
# exploring edition number of all Summer Olympic Games
queryString = """
SELECT DISTINCT ?editions ?name ?ed_number
WHERE {
   ?editions wdt:P31 wd:Q159821 .
   ?editions wdt:P393 ?ed_number .
   # labels
   ?editions sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('ed_number', '3')]
[('editions', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics'), ('ed_number', '11')]
[('editions', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics'), ('ed_number', '17')]
[('editions', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics'), ('ed_number', '32')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('ed_number', '2')]
[('editions', 'http://www.wikidata.org/entity/Q8438'), ('name', '1972 Summer Olympics'), ('ed_number', '20')]
[('editions', 'http://www.wikidata.org/entity/Q8444'), ('name', '1976 Summer Olympics'), ('ed_number', '21')]
[('editions', 'http://www.wikidata.org/entity/Q8470'), ('name', '1988 Summer Olympics'), ('ed_number', '24')]
[('editions', 'http://www.wikidata.org/entity/Q8488'), ('name', '1992 Summer Olympics'), ('ed_number', '25')]
[(

In [36]:
# exploring start time of all Summer Olympic Games
queryString = """
SELECT DISTINCT ?editions ?name ?st_time
WHERE {
   ?editions wdt:P31 wd:Q159821 .
   ?editions wdt:P580 ?st_time .
   # labels
   ?editions sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics'), ('st_time', '1896-04-06T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('st_time', '1900-05-14T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('st_time', '1904-07-01T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games'), ('st_time', '1906-04-22T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics'), ('st_time', '1908-04-27T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics'), ('st_time', '1912-05-05T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8128'), ('name', '1920 Summer Olympics'), ('st_time', '1920-04-20T00:00:00Z')]
[('editions', 'http://www.wikidata.org/entity/Q8132'), ('name', '1924 Summer Olympics'), ('st_time', '1924-

In [37]:
# find all Summer Olympic editions filtering out future editions (wrong)
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {

   ?editions wdt:P31 wd:Q159821 .
   # get the label
   ?editions sc:name ?name.
   FILTER(?name < "2024 Summer Olympics")
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8488'), ('name', '1992 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('editions', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
3


In [8]:
# find all Summer Olympic editions and their countries filtering out future editions (wrong)
queryString = """
SELECT DISTINCT ?editions ?name ?country ?c_name
WHERE {
   ?editions wdt:P31 wd:Q159821 .
   OPTIONAL{?editions wdt:P393 ?ed_number}
   ?editions wdt:P17 ?country .
   # labels
   ?editions sc:name ?name.
   ?country sc:name ?c_name.
   FILTER(?ed_number <= "32")
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q209065'), ('c_name', 'Kingdom of Greece')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q142'), ('c_name', 'France')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q30'), ('c_name', 'United States of America')]
[('editions', 'http://www.wikidata.org/entity/Q8143'), ('name', '1932 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q30'), ('c_name', 'United States of America')]
[('editions', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q1206012'), ('c_name', 'German Reich')]
[('editions', 'http://www.wikidata.org/entity/Q8368'), ('name', '1940 Summer Olympics'), ('country', 'http://www.wikida

In [3]:
# find all Summer Olympic editions and their countries filtering out future editions
queryString = """
SELECT DISTINCT ?editions ?name ?country ?c_name
WHERE {
   ?editions wdt:P31 wd:Q159821 ;
             wdt:P17 ?country ;
             wdt:P580 ?st_time .
   # labels
   ?editions sc:name ?name.
   ?country sc:name ?c_name.
   FILTER(?st_time < NOW())
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q209065'), ('c_name', 'Kingdom of Greece')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q142'), ('c_name', 'France')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q30'), ('c_name', 'United States of America')]
[('editions', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games'), ('country', 'http://www.wikidata.org/entity/Q209065'), ('c_name', 'Kingdom of Greece')]
[('editions', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q174193'), ('c_name', 'United Kingdom of Great Britain and Ireland')]
[('editions', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics'), 

Final query for this task

In [4]:
# find all Summer Olympic editions and their countries filtering out future editions
queryString = """
SELECT DISTINCT ?editions ?name ?country ?c_name
WHERE {
   ?editions wdt:P31 wd:Q159821 ;
             wdt:P17 ?country ;
             wdt:P580 ?st_time .
   # labels
   ?editions sc:name ?name.
   ?country sc:name ?c_name.
   FILTER(?st_time < NOW())
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q209065'), ('c_name', 'Kingdom of Greece')]
[('editions', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q142'), ('c_name', 'France')]
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q30'), ('c_name', 'United States of America')]
[('editions', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games'), ('country', 'http://www.wikidata.org/entity/Q209065'), ('c_name', 'Kingdom of Greece')]
[('editions', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics'), ('country', 'http://www.wikidata.org/entity/Q174193'), ('c_name', 'United Kingdom of Great Britain and Ireland')]
[('editions', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics'), 

## Task 3
Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

In [3]:
# find each Olympic Games category
queryString = """
SELECT DISTINCT ?editions ?name
WHERE {
   ?editions wdt:P279 wd:Q5389 .
   ?editions sc:name ?name .
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('editions', 'http://www.wikidata.org/entity/Q188468'), ('name', 'Ancient Olympic Games')]
[('editions', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('editions', 'http://www.wikidata.org/entity/Q82414'), ('name', 'Winter Olympic Games')]
[('editions', 'http://www.wikidata.org/entity/Q988994'), ('name', 'cancelled Olympic Games')]
4


In [4]:
# find each Olympic Games edition
queryString = """
SELECT DISTINCT ?edition ?name
WHERE {
   {?edition wdt:P31 wd:Q159821} UNION {?edition wdt:P31 wd:Q82414} 
   # labels
   ?edition sc:name ?name
}
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games')]
[('edition', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8123'), ('name', '1916 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8128'), ('name', '1920 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8132'), ('name', '1924 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q9583'), ('name', '1924 Winter Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8138'), ('name', '1928 Summer Olympics')]
[('edition', 'http://

In [5]:
# find all the countries with the #edition they held
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?edition) AS ?total_ed)
WHERE {
   {?edition wdt:P31 wd:Q159821} UNION {?edition wdt:P31 wd:Q82414}
   ?edition wdt:P17 ?country .
   
   #labels
   ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('total_ed', '6')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name',

In [8]:
# find all the countries with the #edition they held, without UNION
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?edition) AS ?total_ed)
WHERE {
   ?edition wdt:P31 ?obj ;
            wdt:P17 ?country .
   FILTER(?obj IN (wd:Q82414, wd:Q159821))
   
   #labels
   ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('total_ed', '6')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name',

In [9]:
# find all the countries with the #edition they held filtering out future editions (wrong)
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?edition) AS ?total_ed)
WHERE {
   {?edition wdt:P31 wd:Q159821} UNION {?edition wdt:P31 wd:Q82414}
   ?edition wdt:P17 ?country .
   OPTIONAL{
       ?edition wdt:P393 ?ed_number
    }            
   
   #labels
   ?country sc:name ?name .
   FILTER(?ed_number <= "32")
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('total_ed', '4')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('name', '

In [9]:
# find all the countries with the #edition they held filtering out future editions without UNION
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?edition) AS ?total_ed)
WHERE {
   ?edition wdt:P31 ?obj ;
            wdt:P17 ?country ;
            wdt:P580 ?st_time .
   FILTER(?obj IN (wd:Q82414, wd:Q159821) && ?st_time < NOW())
   #labels
   ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('total_ed', '5')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name',

Final query for this task

In [10]:
# find all the countries with the #edition they held filtering out future editions without UNION
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?edition) AS ?total_ed)
WHERE {
   ?edition wdt:P31 ?obj ;
            wdt:P17 ?country ;
            wdt:P580 ?st_time .
   FILTER(?obj IN (wd:Q82414, wd:Q159821) && ?st_time < NOW())
   #labels
   ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('total_ed', '3')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('total_ed', '5')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('total_ed', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total_ed', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name',

## Task 4
Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

In [3]:
# properties of 2008 Summer Olympic Games
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8567 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://w

In [4]:
# exploring Commons category (wrong)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q8567 wdt:P373 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# exploring Commons category
queryString = """
SELECT DISTINCT ?obj
WHERE {
   # bind something
   wd:Q8567 wdt:P373 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '2008 Summer Olympics')]
1


In [6]:
# finding all the events of 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   ?p wdt:P361 wd:Q8567 .
   #labels
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q221132'), ('name', 'field hockey at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q387526'), ('name', 'table tennis at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q220654'), ('name', 'wrestling at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q204844'), ('name', 'swimming at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q220942'), ('name', 'water polo at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q7973659'), ('name', "Water polo at the 2008 Summer Olympics – Women's team rosters")]
[('p', 'http://www.wikidata.org/entity/Q212887'), ('name', 'weightlifting at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q220930'), ('name', 'boxing at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q189816'), ('name', 'basketball at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/entity/Q2022469'),

In [7]:
# properties of water polo
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q220942 ?p ?obj .
   #labels
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main c

In [8]:
# finding all the sports of 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?p ?name ?sport ?sp_name
WHERE {
   
   ?p wdt:P361 wd:Q8567 ;
      wdt:P641 ?sport .
   #labels
   ?p sc:name ?name .
   ?sport sc:name ?sp_name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q330719'), ('name', 'triathlon at the 2008 Summer Olympics'), ('sport', 'http://www.wikidata.org/entity/Q10980'), ('sp_name', 'triathlon')]
[('p', 'http://www.wikidata.org/entity/Q318197'), ('name', 'canoeing at the 2008 Summer Olympics'), ('sport', 'http://www.wikidata.org/entity/Q213934'), ('sp_name', 'canoeing and kayaking')]
[('p', 'http://www.wikidata.org/entity/Q2713356'), ('name', 'canoeing at the 2008 Summer Olympics – qualification'), ('sport', 'http://www.wikidata.org/entity/Q213934'), ('sp_name', 'canoeing and kayaking')]
[('p', 'http://www.wikidata.org/entity/Q697280'), ('name', 'artistic gymnastics at the 2008 Summer Olympics'), ('sport', 'http://www.wikidata.org/entity/Q326827'), ('sp_name', 'artistic gymnastics')]
[('p', 'http://www.wikidata.org/entity/Q140307'), ('name', 'synchronized swimming at the 2008 Summer Olympics'), ('sport', 'http://www.wikidata.org/entity/Q180692'), ('sp_name', 'synchronized swimming')]
[('p', 'ht

In [9]:
# finding all the distinct sports of 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?sport ?sp_name
WHERE {
   
   ?p wdt:P361 wd:Q8567 ;
      wdt:P641 ?sport .
   #labels
   ?sport sc:name ?sp_name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q10980'), ('sp_name', 'triathlon')]
[('sport', 'http://www.wikidata.org/entity/Q213934'), ('sp_name', 'canoeing and kayaking')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('sp_name', 'artistic gymnastics')]
[('sport', 'http://www.wikidata.org/entity/Q180692'), ('sp_name', 'synchronized swimming')]
[('sport', 'http://www.wikidata.org/entity/Q465014'), ('sp_name', 'trampolining')]
[('sport', 'http://www.wikidata.org/entity/Q1455'), ('sp_name', 'field hockey')]
[('sport', 'http://www.wikidata.org/entity/Q1734'), ('sp_name', 'volleyball')]
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('sp_name', 'shooting sport')]
[('sport', 'http://www.wikidata.org/entity/Q2736'), ('sp_name', 'association football')]
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('sp_name', 'swimming')]
[('sport', 'http://www.wikidata.org/entity/Q32112'), ('sp_name', 'boxing')]
[('sport', 'http://www.wikidata.org/entity/Q4543'), ('sp_name', 'bea

Final query for this task

In [10]:
# finding all the distinct sports of 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?sport ?sp_name
WHERE {
   
   ?p wdt:P361 wd:Q8567 ;
      wdt:P641 ?sport .
   #labels
   ?sport sc:name ?sp_name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q10980'), ('sp_name', 'triathlon')]
[('sport', 'http://www.wikidata.org/entity/Q213934'), ('sp_name', 'canoeing and kayaking')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('sp_name', 'artistic gymnastics')]
[('sport', 'http://www.wikidata.org/entity/Q180692'), ('sp_name', 'synchronized swimming')]
[('sport', 'http://www.wikidata.org/entity/Q465014'), ('sp_name', 'trampolining')]
[('sport', 'http://www.wikidata.org/entity/Q1455'), ('sp_name', 'field hockey')]
[('sport', 'http://www.wikidata.org/entity/Q1734'), ('sp_name', 'volleyball')]
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('sp_name', 'shooting sport')]
[('sport', 'http://www.wikidata.org/entity/Q2736'), ('sp_name', 'association football')]
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('sp_name', 'swimming')]
[('sport', 'http://www.wikidata.org/entity/Q32112'), ('sp_name', 'boxing')]
[('sport', 'http://www.wikidata.org/entity/Q4543'), ('sp_name', 'bea

## Task 5
Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

In [3]:
# finding properties of tennis
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q847 ?p ?obj .
   #labels
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main

In [8]:
# finding subjects for "part of" tennis
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P361 wd:Q847 .
   #labels
   ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [9]:
# finding subjects for "instance of" tennis
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P31 wd:Q847 .
   #labels
   ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q70062392'), ('name', '2019 Rolex Shanghai Masters – doubles')]
1


In [10]:
# finding subjects for "part of" tennis at 2008 Olympics
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P361 wd:Q194246 .
   #labels
   ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2024971'), ('name', 'tennis at the 2008 Summer Olympics – qualification')]
[('s', 'http://www.wikidata.org/entity/Q220866'), ('name', "tennis at the 2008 Summer Olympics – women's doubles")]
[('s', 'http://www.wikidata.org/entity/Q752413'), ('name', "tennis at the 2008 Summer Olympics – women's singles")]
[('s', 'http://www.wikidata.org/entity/Q752428'), ('name', "tennis at the 2008 Summer Olympics – men's singles")]
[('s', 'http://www.wikidata.org/entity/Q757054'), ('name', "tennis at the 2008 Summer Olympics – men's doubles")]
5


In [12]:
# properties of tennis 2008 women's doubles
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q220866 ?p ?obj .
   #labels
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9055'), ('name', 'Olympedia event I

In [15]:
# finding subjects for "part of" of each 2008 Olympics sport removing qualifications
queryString = """
SELECT DISTINCT ?event ?name ?d_name
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event .
   #labels
   ?event sc:name ?name .
   ?discipline sc:name ?d_name .
   FILTER(!regex(?d_name, "qualification$"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('event', 'http://www.wikidata.org/entity/Q140307'), ('name', 'synchronized swimming at the 2008 Summer Olympics'), ('d_name', "synchronized swimming at the 2008 Summer Olympics – women's team")]
[('event', 'http://www.wikidata.org/entity/Q318197'), ('name', 'canoeing at the 2008 Summer Olympics'), ('d_name', "canoeing at the 2008 Summer Olympics – women's slalom K-1")]
[('event', 'http://www.wikidata.org/entity/Q223781'), ('name', 'judo at the 2008 Summer Olympics'), ('d_name', "judo at the 2008 Summer Olympics – women's 57 kg")]
[('event', 'http://www.wikidata.org/entity/Q223519'), ('name', 'archery at the 2008 Summer Olympics'), ('d_name', "archery at the 2008 Summer Olympics – women's team")]
[('event', 'http://www.wikidata.org/entity/Q189941'), ('name', 'athletics at the 2008 Summer Olympics'), ('d_name', "athletics at the 2008 Summer Olympics – men's discus throw")]
[('event', 'http://www.wikidata.org/entity/Q189941'), ('name', 'athletics at the 2008 Summer Olympics'), (

In [20]:
# counting the discipline for each sport
queryString = """
SELECT DISTINCT ?sport ?name COUNT((?discipline) AS ?disciplines)
WHERE {
   ?event wdt:P361 wd:Q8567 ;
          wdt:P641 ?sport .
   ?discipline wdt:P361 ?event .
   #labels
   ?sport sc:name ?name .
   ?discipline sc:name ?d_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?sport ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('name', 'shooting sport'), ('callret-2', '15')]
[('sport', 'http://www.wikidata.org/entity/Q847'), ('name', 'tennis'), ('callret-2', '4')]
[('sport', 'http://www.wikidata.org/entity/Q171038'), ('name', 'softball'), ('callret-2', '1')]
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('name', 'equestrian sport'), ('callret-2', '6')]
[('sport', 'http://www.wikidata.org/entity/Q12100'), ('name', 'fencing'), ('callret-2', '10')]
[('sport', 'http://www.wikidata.org/entity/Q14085739'), ('name', 'sailing'), ('callret-2', '11')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('name', 'artistic gymnastics'), ('callret-2', '14')]
[('sport', 'http://www.wikidata.org/entity/Q36389'), ('name', 'taekwondo'), ('callret-2', '8')]
[('sport', 'http://www.wikidata.org/entity/Q43450'), ('name', 'gymnastics'), ('callret-2', '3')]
[('sport', 'http://www.wikidata.org/entity/Q465014'), ('name', 'trampolining'), ('callret-2', '2')]
[('s

Final query for this task

In [21]:
# counting the discipline for each sport
queryString = """
SELECT DISTINCT ?sport ?name COUNT((?discipline) AS ?disciplines)
WHERE {
   ?event wdt:P361 wd:Q8567 ;
          wdt:P641 ?sport .
   ?discipline wdt:P361 ?event .
   #labels
   ?sport sc:name ?name .
   ?discipline sc:name ?d_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?sport ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('name', 'shooting sport'), ('callret-2', '15')]
[('sport', 'http://www.wikidata.org/entity/Q847'), ('name', 'tennis'), ('callret-2', '4')]
[('sport', 'http://www.wikidata.org/entity/Q171038'), ('name', 'softball'), ('callret-2', '1')]
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('name', 'equestrian sport'), ('callret-2', '6')]
[('sport', 'http://www.wikidata.org/entity/Q12100'), ('name', 'fencing'), ('callret-2', '10')]
[('sport', 'http://www.wikidata.org/entity/Q14085739'), ('name', 'sailing'), ('callret-2', '11')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('name', 'artistic gymnastics'), ('callret-2', '14')]
[('sport', 'http://www.wikidata.org/entity/Q36389'), ('name', 'taekwondo'), ('callret-2', '8')]
[('sport', 'http://www.wikidata.org/entity/Q43450'), ('name', 'gymnastics'), ('callret-2', '3')]
[('sport', 'http://www.wikidata.org/entity/Q465014'), ('name', 'trampolining'), ('callret-2', '2')]
[('s

## Task 6
Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

In [3]:
# exploring winner property of tennis 2008 women's doubles
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q220866 wdt:P1346 ?obj .
   #labels
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams')]
1


In [4]:
# returning the winner for each discipline in 2008 excluding qualifications
queryString = """
SELECT DISTINCT ?d_name ?athlete ?name
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   FILTER(!regex(?d_name, "qualification$"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('d_name', "cycling at the 2008 Summer Olympics – women's points race"), ('athlete', 'http://www.wikidata.org/entity/Q260825'), ('name', 'Leire Olaberria')]
[('d_name', "tennis at the 2008 Summer Olympics – men's doubles"), ('athlete', 'http://www.wikidata.org/entity/Q193661'), ('name', 'Stan Wawrinka')]
[('d_name', 'boxing at the 2008 Summer Olympics – welterweight'), ('athlete', 'http://www.wikidata.org/entity/Q560365'), ('name', 'Bakhyt Sarsekbayev')]
[('d_name', "athletics at the 2008 Summer Olympics – women's 200 metres"), ('athlete', 'http://www.wikidata.org/entity/Q5796'), ('name', 'Veronica Campbell-Brown')]
[('d_name', "cycling at the 2008 Summer Olympics – men's points race"), ('athlete', 'http://www.wikidata.org/entity/Q60822'), ('name', 'Roger Kluge')]
[('d_name', "gymnastics at the 2008 Summer Olympics – men's vault"), ('athlete', 'http://www.wikidata.org/entity/Q499924'), ('name', 'Leszek Blanik')]
[('d_name', "cycling at the 2008 Summer Olympics – men's cross-co

In [5]:
# returning all the athletes that won a gold medal in 2008 with the #golds
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT(*) AS ?golds)
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q560365'), ('name', 'Bakhyt Sarsekbayev'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q499924'), ('name', 'Leszek Blanik'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q666580'), ('name', 'Nino Schurter'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q270657'), ('name', 'Anna Meares'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q6419'), ('name', 'Barbora Špotáková'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q483244'), ('name', 'Lee Yong-dae'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q234495'), ('name', 'Maurren Maggi'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q140387'), ('name', 'Germany at the 2008 Summer Olympics'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q221185'), ('name', 'Kateřina Emmons'), ('golds', '1')]
[('athlete', 'http://www.wikidata.org/entity/Q449073'), ('name', 'Ben Ain

In [6]:
# returning the top 10 athletes that won the most gold medals in 2008
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT(*) AS ?golds)
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
ORDER BY DESC(?golds)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('golds', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('golds', '3')]
[('athlete', 'http://www.wikidata.org/entity/Q312690'), ('name', 'Kosuke Kitajima'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q270693'), ('name', 'Guo Jingjing'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q9119'), ('name', 'Kenenisa Bekele Beyecha'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q228660'), ('name', 'Stephanie Rice'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q57273'), ('name', 'Britta Steffen'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q49287'), ('name', 'Rebecca Adlington'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q227015'), ('name', 'Zou Kai'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt'), ('golds', '2')]
10


In [12]:
# explore Human node
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    ?s ?p wd:Q5 .
    #labels
    ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1605'), ('name', 'has natural reservoir')]
[('p', 'http://www.wikidata.org/prop/direct/P6962'), ('name', 'narrative motif')]
[('p', 'http://www.wikidata.org/prop/direct/P4675'), ('name', 'appears in the form of')]
[('p', 'http://www.wikidata.org/prop/direct/P1034'), ('name', 'main food source')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('name', 'chief executive officer')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org

In [13]:
# exploring instance of Human
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    ?s wdt:P31 wd:Q5 .
    #labels
    ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto')]
[('s', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek')]
[('s', 'http://www.wikidata.org/entity/Q102115260'), ('name', 'Kojiro Kobayashi')]
[('s', 'http://www.wikidata.org/entity/Q102118153'), ('name', 'Miguel E. M. Herrera')]
[('s', 'http://www.wikidata.org/entity/Q105753880'), ('name', 'Mateo Egea')]
[('s', 'http://www.wikidata.org/entity/Q107520918'), ('name', 'Jerónimo Latorre')]
[('s', 'http://www.wikidata.org/entity/Q11404285'), ('name', 'Mikiko Kitano')]
[('s', 'http://www.wikidata.org/entity/Q11520274'), ('name', 'Honjō Fusanaga')]
[('s', 'http://www.wikidata.org/entity/Q11562694'), ('name', 'Takayoshi Watanabe')]
[('s', 'http://www.wikidata.org/entity/Q11685026'), ('name', 'Joan de Gènova')]
[('s', 'http://www.wikidata.org/entity/Q11686949'), ('name', 'José Ximénez')]
[('s', 'http://www.wikidata.org/entity/Q11871976'), ('name', 'Klaus Wirzenius')]
[('s', 'http://www.w

In [14]:
# returning the top 10 athletes that won the most gold medals in 2008 (check if it's a human and not a team)
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT(*) AS ?golds)
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
ORDER BY DESC(?golds)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('golds', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('golds', '3')]
[('athlete', 'http://www.wikidata.org/entity/Q312690'), ('name', 'Kosuke Kitajima'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q270693'), ('name', 'Guo Jingjing'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q9119'), ('name', 'Kenenisa Bekele Beyecha'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q228660'), ('name', 'Stephanie Rice'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q57273'), ('name', 'Britta Steffen'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q49287'), ('name', 'Rebecca Adlington'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q227015'), ('name', 'Zou Kai'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt'), ('golds', '2')]
10


Final query for this task

In [15]:
# returning the top 10 athletes that won the most gold medals in 2008
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT(*) AS ?golds)
WHERE {
   ?event wdt:P361 wd:Q8567 .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
ORDER BY DESC(?golds)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('golds', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('golds', '3')]
[('athlete', 'http://www.wikidata.org/entity/Q312690'), ('name', 'Kosuke Kitajima'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q270693'), ('name', 'Guo Jingjing'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q9119'), ('name', 'Kenenisa Bekele Beyecha'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q228660'), ('name', 'Stephanie Rice'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q57273'), ('name', 'Britta Steffen'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q49287'), ('name', 'Rebecca Adlington'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q227015'), ('name', 'Zou Kai'), ('golds', '2')]
[('athlete', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt'), ('golds', '2')]
10


## Task 7
Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

In [10]:
# returning the athletes that won a gold medal with each edition
queryString = """
SELECT DISTINCT ?athlete ?name ?ed_name
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
ORDER BY DESC(?ed_name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q20680380'), ('name', 'Yulimar Rojas'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q20035636'), ('name', 'Elaine Thompson-Herah'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q19974493'), ('name', 'Jakob Ingebrigtsen'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q6399'), ('name', 'Anita Włodarczyk'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q1739'), ('name', 'Mutaz Essa Barshim'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q454772'), ('name', 'Mariya Kuchina'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q6370'), ('name', 'Gong Lijiao'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q3012723'), ('name', 'Damian Warner'), ('ed_name', '2020 Summer Olympics')]
[('athlete'

In [16]:
# returning the athletes that won a gold medal with each edition (check if it's a Human and not a team)
queryString = """
SELECT DISTINCT ?athlete ?name ?ed_name
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
ORDER BY DESC(?ed_name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q20680380'), ('name', 'Yulimar Rojas'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q20035636'), ('name', 'Elaine Thompson-Herah'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q19974493'), ('name', 'Jakob Ingebrigtsen'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q6399'), ('name', 'Anita Włodarczyk'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q1739'), ('name', 'Mutaz Essa Barshim'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q454772'), ('name', 'Mariya Kuchina'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q6370'), ('name', 'Gong Lijiao'), ('ed_name', '2020 Summer Olympics')]
[('athlete', 'http://www.wikidata.org/entity/Q3012723'), ('name', 'Damian Warner'), ('ed_name', '2020 Summer Olympics')]
[('athlete'

In [17]:
# returning the athletes that won a gold medal with the number of editions
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT(?ed_name) AS ?editions)
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
ORDER BY DESC(?editions)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('editions', '14')]
[('athlete', 'http://www.wikidata.org/entity/Q460643'), ('name', 'Jason Kenny'), ('editions', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q193876'), ('name', 'Bradley Wiggins'), ('editions', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q52674'), ('name', 'Ray Ewry'), ('editions', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q131237'), ('name', 'Carl Lewis'), ('editions', '7')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('editions', '7')]
[('athlete', 'http://www.wikidata.org/entity/Q137374'), ('name', 'Katie Ledecky'), ('editions', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q1017119'), ('name', 'Burton Downing'), ('editions', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q312684'), ('name', 'Sawao Katō'), ('editions', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q219309'), ('name', 'Nikolai Andrianov'), 

In [19]:
# returning the athletes that won a gold medal with the number of distinct editions
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT (DISTINCT ?ed_name) AS ?editions)
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
ORDER BY DESC(?editions)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q312971'), ('name', 'Aladár Gerevich'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q193876'), ('name', 'Bradley Wiggins'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q52642'), ('name', 'Al Oerter'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q270657'), ('name', 'Anna Meares'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q462790'), ('name', 'Mijaín López'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q352640'), ('name', 'Edoardo Mangiarotti'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q66229'), ('name', 'Jens Fiedler'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q131237'), ('name', 'Carl Lewis'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q460643'), ('name', 'Jason Kenny'), 

In [3]:
# returning the athletes that won a gold medal in at least 4 distinct editions
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT (DISTINCT ?ed_name) AS ?editions)
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
HAVING (COUNT(DISTINCT ?ed_name) >= 4)
ORDER BY DESC(?editions)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q312971'), ('name', 'Aladár Gerevich'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q193876'), ('name', 'Bradley Wiggins'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q52642'), ('name', 'Al Oerter'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q270657'), ('name', 'Anna Meares'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q462790'), ('name', 'Mijaín López'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q352640'), ('name', 'Edoardo Mangiarotti'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q66229'), ('name', 'Jens Fiedler'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q131237'), ('name', 'Carl Lewis'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q460643'), ('name', 'Jason Kenny'), 

Final query for this task

In [4]:
# returning the athletes that won a gold medal in at least 4 distinct editions
queryString = """
SELECT DISTINCT ?athlete ?name (COUNT (DISTINCT ?ed_name) AS ?editions)
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?name .
   ?edition sc:name ?ed_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?name
HAVING (COUNT(DISTINCT ?ed_name) >= 4)
ORDER BY DESC(?editions)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q312971'), ('name', 'Aladár Gerevich'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q193876'), ('name', 'Bradley Wiggins'), ('editions', '5')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q52642'), ('name', 'Al Oerter'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q270657'), ('name', 'Anna Meares'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q462790'), ('name', 'Mijaín López'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q352640'), ('name', 'Edoardo Mangiarotti'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q66229'), ('name', 'Jens Fiedler'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q131237'), ('name', 'Carl Lewis'), ('editions', '4')]
[('athlete', 'http://www.wikidata.org/entity/Q460643'), ('name', 'Jason Kenny'), 

## Task 8
Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal).

In [4]:
# find all the athletes with the number of gold medals they have
queryString = """
SELECT DISTINCT ?athlete ?a_name (COUNT(*) AS ?golds)
WHERE {
   ?edition wdt:P31 wd:Q159821 .
   ?event wdt:P361 ?edition .
   ?discipline wdt:P361 ?event ;
               wdt:P1346 ?athlete .
               
   # check if Human               
   ?athlete wdt:P31 wd:Q5 .
   #labels
   ?discipline sc:name ?d_name .
   ?athlete sc:name ?a_name .
   FILTER(!regex(?d_name, "qualification$"))
}
GROUP BY ?athlete ?a_name
ORDER BY DESC(?golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q39562'), ('a_name', 'Michael Phelps'), ('golds', '14')]
[('athlete', 'http://www.wikidata.org/entity/Q460643'), ('a_name', 'Jason Kenny'), ('golds', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q193876'), ('a_name', 'Bradley Wiggins'), ('golds', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q52674'), ('a_name', 'Ray Ewry'), ('golds', '8')]
[('athlete', 'http://www.wikidata.org/entity/Q131237'), ('a_name', 'Carl Lewis'), ('golds', '7')]
[('athlete', 'http://www.wikidata.org/entity/Q310383'), ('a_name', 'Chris Hoy'), ('golds', '7')]
[('athlete', 'http://www.wikidata.org/entity/Q137374'), ('a_name', 'Katie Ledecky'), ('golds', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q312684'), ('a_name', 'Sawao Katō'), ('golds', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q219309'), ('a_name', 'Nikolai Andrianov'), ('golds', '6')]
[('athlete', 'http://www.wikidata.org/entity/Q352640'), ('a_name', 'Edoardo Mangiarotti'), ('g

In [7]:
# find all the athletes with the number of gold medals they have divided by sport
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {        
    {
        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete ;
                       wdt:P641 ?sport .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?athlete ?sport
    }
    
    ?sport sc:name ?s_name .
    ?athlete sc:name ?a_name .
}
GROUP BY ?sport ?s_name
ORDER BY (?sport)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q1031445'), ('s_name', 'cross-country cycling'), ('athlete', 'http://www.wikidata.org/entity/Q2619455'), ('a_name', 'Carlos Coloma Nicolás'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q1031445'), ('s_name', 'cross-country cycling'), ('athlete', 'http://www.wikidata.org/entity/Q16562637'), ('a_name', 'Jenny Rissveds'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q1031445'), ('s_name', 'cross-country cycling'), ('athlete', 'http://www.wikidata.org/entity/Q554522'), ('a_name', 'Jaroslav Kulhavý'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q1031445'), ('s_name', 'cross-country cycling'), ('athlete', 'http://www.wikidata.org/entity/Q666580'), ('a_name', 'Nino Schurter'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q1031445'), ('s_name', 'cross-country cycling'), ('athlete', 'http://www.wikidata.org/entity/Q271005'), ('a_name', 'Maja Włoszczowska'), ('golds', '1')]
[('sport', 'h

In [15]:
# find all the athletes with the number of gold medals they have divided by sport (wrong)
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (MAX(?golds) AS ?max_golds)
WHERE {        
    {
        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete ;
                       wdt:P641 ?sport .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?athlete ?sport
    }
    
    ?sport sc:name ?s_name .
    ?athlete sc:name ?a_name .
}
GROUP BY ?sport ?s_name ?athlete ?a_name
ORDER BY DESC(?max_golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('s_name', 'swimming'), ('athlete', 'http://www.wikidata.org/entity/Q39562'), ('a_name', 'Michael Phelps'), ('max_golds', '14')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('athlete', 'http://www.wikidata.org/entity/Q52674'), ('a_name', 'Ray Ewry'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q460643'), ('a_name', 'Jason Kenny'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q193876'), ('a_name', 'Bradley Wiggins'), ('max_golds', '7')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('athlete', 'http://www.wikidata.org/entity/Q131237'), ('a_name', 'Carl Lewis'), ('max_golds', '7')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('

In [3]:
# find all the athletes with the number of gold medals they have divided by sport (wrong)
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {

    ?discipline wdt:P1346 ?athlete ;
                wdt:P641 ?sport .
        
    FILTER(?sport = ?sp && ?golds = ?max_golds)
    {
        SELECT DISTINCT ?sp (MAX(COUNT(DISTINCT (?athlete)) AS ?golds) as ?max_golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete ;
                       wdt:P641 ?sp .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?sp
    }
    UNION
    {
        SELECT DISTINCT ?athlete (COUNT(*) AS ?golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?athlete
    }
    
    ?athlete sc:name ?a_name .
    ?sport sc:name ?s_name .
}
ORDER BY DESC(?golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [12]:
# find the maximum number of golds won by a single athelete divided by sport
queryString = """
SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
WHERE {        
    {
        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete ;
                       wdt:P641 ?sport .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?sport ?athlete
    }
    ?sport sc:name ?s_name .
}
GROUP BY ?sport ?s_name 
ORDER BY DESC(?max_golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('s_name', 'swimming'), ('max_golds', '14')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q12100'), ('s_name', 'fencing'), ('max_golds', '6')]
[('sport', 'http://www.wikidata.org/entity/Q43450'), ('s_name', 'gymnastics'), ('max_golds', '6')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('s_name', 'artistic gymnastics'), ('max_golds', '5')]
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('s_name', 'shooting sport'), ('max_golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q14085739'), ('s_name', 'sailing'), ('max_golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('s_name', 'equestrian sport'), ('max_golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q108429'), ('s_name', 'archery'), ('max

In [14]:
# find all the athletes with the number of gold medals they have divided by sport (wrong)
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {
    {
        SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
        WHERE {        
            {
                SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                WHERE {
                   ?edition wdt:P31 wd:Q159821 .
                   ?event wdt:P361 ?edition .
                   ?discipline wdt:P361 ?event ;
                               wdt:P1346 ?athlete ;
                               wdt:P641 ?sport .

                   # check if Human               
                   ?athlete wdt:P31 wd:Q5 .
                   #labels
                   ?discipline sc:name ?d_name .
                   FILTER(!regex(?d_name, "qualification$"))
                }
                GROUP BY ?sport ?athlete
            }
            ?sport sc:name ?s_name .
        }
        GROUP BY ?sport ?s_name 
    }
    UNION
    {
        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
        WHERE {
           ?edition wdt:P31 wd:Q159821 .
           ?event wdt:P361 ?edition .
           ?discipline wdt:P361 ?event ;
                       wdt:P1346 ?athlete ;
                       wdt:P641 ?sport .

           # check if Human               
           ?athlete wdt:P31 wd:Q5 .
           #labels
           ?discipline sc:name ?d_name .
           FILTER(!regex(?d_name, "qualification$"))
        }
        GROUP BY ?athlete ?sport
    }
    FILTER (?golds = ?max_golds)
    ?athlete sc:name ?a_name .
}
ORDER BY DESC(?golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [2]:
# find all the athletes with the number of gold medals they have divided by sport, with max_golds that is the maximum number of golds
# for that specific sport
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (COUNT(*) as ?golds) ?max_golds
WHERE {
    ?edition wdt:P31 wd:Q159821 .
    ?event wdt:P361 ?edition .
    ?discipline wdt:P361 ?event ;
                wdt:P1346 ?athlete ;
                wdt:P641 ?sport .
    ?athlete sc:name ?a_name .
    {
        SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
        WHERE {        
            {
                SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                WHERE {
                   ?edition wdt:P31 wd:Q159821 .
                   ?event wdt:P361 ?edition .
                   ?discipline wdt:P361 ?event ;
                               wdt:P1346 ?athlete ;
                               wdt:P641 ?sport .

                   # check if Human               
                   ?athlete wdt:P31 wd:Q5 .
                   #labels
                   ?discipline sc:name ?d_name .
                   FILTER(!regex(?d_name, "qualification$"))
                }
                GROUP BY ?sport ?athlete
            }
            ?sport sc:name ?s_name .
        }
        GROUP BY ?sport ?s_name
    }   
}
GROUP BY ?sport ?s_name ?athlete ?a_name ?max_golds
ORDER BY DESC(?golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('s_name', 'swimming'), ('athlete', 'http://www.wikidata.org/entity/Q39562'), ('golds', '14'), ('max_golds', '14')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q460643'), ('golds', '8'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('athlete', 'http://www.wikidata.org/entity/Q52674'), ('golds', '8'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q310383'), ('golds', '7'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q193876'), ('golds', '7'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('athlete', 'http://www.wikidata.org/entity/Q131237'), ('go

In [4]:
# find all the athletes with the number of gold medals they have divided by sport, filtering the ones that have golds=max_golds
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (COUNT(*) as ?golds) ?max_golds
WHERE {
    ?edition wdt:P31 wd:Q159821 .
    ?event wdt:P361 ?edition .
    ?discipline wdt:P361 ?event ;
                wdt:P1346 ?athlete ;
                wdt:P641 ?sport .
    ?athlete sc:name ?a_name .
    {
        SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
        WHERE {        
            {
                SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                WHERE {
                   ?edition wdt:P31 wd:Q159821 .
                   ?event wdt:P361 ?edition .
                   ?discipline wdt:P361 ?event ;
                               wdt:P1346 ?athlete ;
                               wdt:P641 ?sport .

                   # check if Human               
                   ?athlete wdt:P31 wd:Q5 .
                   #labels
                   ?discipline sc:name ?d_name .
                   FILTER(!regex(?d_name, "qualification$"))
                }
                GROUP BY ?sport ?athlete
            }
            ?sport sc:name ?s_name .
        }
        GROUP BY ?sport ?s_name
    }   
}
GROUP BY ?sport ?s_name ?athlete ?a_name ?max_golds
HAVING(COUNT(*) = ?max_golds)
ORDER BY DESC(?golds)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('s_name', 'swimming'), ('athlete', 'http://www.wikidata.org/entity/Q39562'), ('a_name', 'Michael Phelps'), ('golds', '14'), ('max_golds', '14')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s_name', 'athletics'), ('athlete', 'http://www.wikidata.org/entity/Q52674'), ('a_name', 'Ray Ewry'), ('golds', '8'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q221635'), ('s_name', 'track cycling'), ('athlete', 'http://www.wikidata.org/entity/Q460643'), ('a_name', 'Jason Kenny'), ('golds', '8'), ('max_golds', '8')]
[('sport', 'http://www.wikidata.org/entity/Q43450'), ('s_name', 'gymnastics'), ('athlete', 'http://www.wikidata.org/entity/Q219309'), ('a_name', 'Nikolai Andrianov'), ('golds', '6'), ('max_golds', '6')]
[('sport', 'http://www.wikidata.org/entity/Q43450'), ('s_name', 'gymnastics'), ('athlete', 'http://www.wikidata.org/entity/Q312684'), ('a_name', 'Sawao Katō'), ('golds', '6'), ('max_golds', '6')]
[('

In [5]:
# find all the athletes with the number of gold medals they have divided by sport (ordered by sport, and removing max_golds)
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {
    FILTER(?golds = ?max_golds)
    {
        SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (COUNT(*) as ?golds) ?max_golds
        WHERE {
            ?edition wdt:P31 wd:Q159821 .
            ?event wdt:P361 ?edition .
            ?discipline wdt:P361 ?event ;
                        wdt:P1346 ?athlete ;
                        wdt:P641 ?sport .
            ?athlete sc:name ?a_name .
            {
                SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
                WHERE {        
                    {
                        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                        WHERE {
                           ?edition wdt:P31 wd:Q159821 .
                           ?event wdt:P361 ?edition .
                           ?discipline wdt:P361 ?event ;
                                       wdt:P1346 ?athlete ;
                                       wdt:P641 ?sport .

                           # check if Human               
                           ?athlete wdt:P31 wd:Q5 .
                           #labels
                           ?discipline sc:name ?d_name .
                           FILTER(!regex(?d_name, "qualification$"))
                        }
                        GROUP BY ?sport ?athlete
                    }
                    ?sport sc:name ?s_name .
                }
                GROUP BY ?sport ?s_name
            }   
        }
        GROUP BY ?sport ?s_name ?athlete ?a_name ?max_golds
    }
}
ORDER BY DESC(?sport)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('s_name', 'equestrian sport'), ('athlete', 'http://www.wikidata.org/entity/Q262338'), ('a_name', 'Nicole Uphoff'), ('golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q847'), ('s_name', 'tennis'), ('athlete', 'http://www.wikidata.org/entity/Q11459'), ('a_name', 'Serena Williams'), ('golds', '3')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q107351542'), ('a_name', 'Keegan Palmer'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q56486600'), ('a_name', 'Sakura Yosozumi'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q66363755'), ('a_name', 'Yūto Horigome'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboa

Final query for this task

In [3]:
# find the athletes that have won more gold medals divided by sport
# this is a quite complex solution, but I couldn't manage to find an easier one
# it also returns multiple athletes for each sport since it could happen that they have the same amount of gold medals
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {
    FILTER(?golds = ?max_golds)
    {
        SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (COUNT(*) as ?golds) ?max_golds
        WHERE {
            ?edition wdt:P31 wd:Q159821 .
            ?event wdt:P361 ?edition .
            ?discipline wdt:P361 ?event ;
                        wdt:P1346 ?athlete ;
                        wdt:P641 ?sport .
            ?athlete sc:name ?a_name .
            {
                SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
                WHERE {        
                    {
                        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                        WHERE {
                           ?edition wdt:P31 wd:Q159821 .
                           ?event wdt:P361 ?edition .
                           ?discipline wdt:P361 ?event ;
                                       wdt:P1346 ?athlete ;
                                       wdt:P641 ?sport .

                           # check if Human               
                           ?athlete wdt:P31 wd:Q5 .
                           #labels
                           ?discipline sc:name ?d_name .
                           FILTER(!regex(?d_name, "qualification$"))
                        }
                        GROUP BY ?sport ?athlete
                    }
                    ?sport sc:name ?s_name .
                }
                GROUP BY ?sport ?s_name
            }   
        }
        GROUP BY ?sport ?s_name ?athlete ?a_name ?max_golds
    }
}
ORDER BY DESC(?sport)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('s_name', 'equestrian sport'), ('athlete', 'http://www.wikidata.org/entity/Q262338'), ('a_name', 'Nicole Uphoff'), ('golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q847'), ('s_name', 'tennis'), ('athlete', 'http://www.wikidata.org/entity/Q11459'), ('a_name', 'Serena Williams'), ('golds', '3')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q107351542'), ('a_name', 'Keegan Palmer'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q56486600'), ('a_name', 'Sakura Yosozumi'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q66363755'), ('a_name', 'Yūto Horigome'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboa